In [3]:

from dotenv import load_dotenv
import openai
import os 

import random


import pandas as pd


In [2]:
load_dotenv()
openai.organization = os.environ['OPENAI_ORG']
openai.api_key = os.environ['OPENAI_API_KEY']

In [13]:
df = pd.read_csv('./../../data/wyscout_tabular/germany.csv')

In [10]:
teams = pd.read_json('./../../data/wyscout/Teams/teams.json')
matches = pd.read_json('./../../data/wyscout/Matches/matches_Germany.json')
players = pd.read_json('./../../data/wyscout/players.json')

In [14]:
df.columns

Index(['Unnamed: 0', 'eventId', 'subEventName', 'playerId', 'matchId',
       'eventName', 'teamId', 'matchPeriod', 'eventSec', 'subEventId', 'id',
       'x_inicio', 'y_inicio', 'x_fin', 'y_fin', 'tag_0', 'tag_1', 'tag_2',
       'tag_3', 'tag_4', 'tag_5'],
      dtype='object')

In [17]:
teams.columns

Index(['city', 'name', 'wyId', 'officialName', 'area', 'type'], dtype='object')

In [18]:
players.columns

Index(['passportArea', 'weight', 'firstName', 'middleName', 'lastName',
       'currentTeamId', 'birthDate', 'height', 'role', 'birthArea', 'wyId',
       'foot', 'shortName', 'currentNationalTeamId'],
      dtype='object')

In [16]:
df = df.merge(teams, left_on='teamId', right_on='wyId', how='left')

In [19]:
df  = df.merge(players, left_on='playerId', right_on ='wyId', how='left')

In [20]:
def create_prompt(event):
    
    team = str(event['name']).encode('utf-8').decode('unicode_escape')
    player = str(event['shortName']).encode('utf-8').decode('unicode_escape')
    event_type = event['eventName']
    
    game_sec = int(event['eventSec']%60)
    game_min = int(event['eventSec']/60)
    half = event['matchPeriod']
    
    if game_sec < 10:
        game_sec = "0"+str(game_sec)
    if game_min < 10:
        game_min = "0"+str(game_min)
    if half == '2H':
        game_min +=45
        
    prompt = f"""Data of the event:
    - player: {player}
    - team: {team}
    - action: {event_type}
    - game time: {game_min}:{game_sec}
    
    Rewrite the event as a tweet
    """
        
    return prompt
    
    

In [23]:
create_prompt(df.iloc[0])

'Data of the event:\n    - player: K. Volland\n    - team: Bayer Leverkusen\n    - action: Pass\n    - game time: 00:02\n    \n    Rewrite the event as a tweet\n    '

In [25]:
df.iloc[0].to_string()

"Unnamed: 0                                                               0\neventId                                                                  8\nsubEventName                                                   Simple pass\nplayerId                                                             15231\nmatchId                                                            2516739\neventName                                                             Pass\nteamId                                                                2446\nmatchPeriod                                                             1H\neventSec                                                          2.409746\nsubEventId                                                            85.0\nid                                                               179896442\nx_inicio                                                                50\ny_inicio                                                                50\nx_fin      

In [27]:
teams['name'].unique()

array(['Newcastle United', 'Celta de Vigo', 'Espanyol',
       'Deportivo Alav\\u00e9s', 'Levante', 'Troyes', 'Getafe',
       "Borussia M'gladbach", 'Huddersfield Town', 'Athletic Club',
       'Atl\\u00e9tico Madrid', 'Olympique Lyonnais', 'PSG', 'Valencia',
       'Real Madrid', 'Barcelona', 'Las Palmas', 'Legan\\u00e9s', 'SPAL',
       'Swansea City', 'Olympique Marseille', 'Nantes', 'Nice', 'Rennes',
       'Strasbourg', 'Eibar', 'AFC Bournemouth', 'Brighton & Hove Albion',
       'Werder Bremen', 'Bayer Leverkusen', 'Borussia Dortmund',
       'Bayern M\\u00fcnchen', 'Stuttgart', 'Schalke 04', 'Milan',
       'Angers', 'Juventus', 'Roma', 'Sassuolo', 'Burnley', 'Bordeaux',
       'Hannover 96', 'Dijon', 'Hertha BSC', 'Wolfsburg', 'Hamburger SV',
       'Freiburg', 'Bologna', 'Metz', 'Sampdoria', 'Chievo', 'Lazio',
       'Udinese', 'Internazionale', 'Leicester City', 'West Ham United',
       'Stoke City', 'Benevento', 'Saint-\\u00c9tienne', 'Girona',
       'Watford', 'Hoffenhei